# Priprema za prvi računarski kolokvijum
Dat je skup podataka o zaposlenima. Cilj je napraviti model linearne regresije koji na osnovu podatak iz data/train.csv fajla predviđa platu.

Kao metriku u zadacima koristiti *RMSE*, osim ako nije drugačije naznačeno.

Uzeti pouzdanost od 95% za sve statističke testove, osim ako nije drugačije naznačeno.

**Zadatak 1.** <br>
Napraviti model linearne regresije koji koristi sve atribute. 
Ispisati meru koju model ostvaruje.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from utils_nans1 import *

# učitavamo podatke
df = pd.read_csv('data/train.csv', sep=',')
df.head()

,plata,zvanje,oblast,godina_doktor,godina_iskustva,pol Zenski,pol Muski
0,77500,1.0,0,NaN,2,1,0
1,142253,3.0,1,49.0,60,0,1
2,124714,NaN,1,23.0,15,0,1
3,147765,3.0,1,45.0,45,0,1
4,106294,NaN,0,21.0,8,0,1


In [2]:
# Pre treniranja modela moramo rešiti nedostajuće vrednosti.
# Najlakši način je da izbacimo sve redove.
df = df.dropna()

# fitujemo model (tražimo parametre)
x = df.drop(columns=['plata'])
y = df['plata']
model = get_fitted_model(x, y)

# prijavljujemo metriku na test skupu
df_test = pd.read_csv('data/test.csv', sep=',')
x_test = df_test.drop(columns=['plata'])
y_test = df_test['plata']
test_rmse = get_rmse(model, x_test, y_test)
print(test_rmse)

11845.08209104146


**Zadatak 2.** <br>
Kada zaposleni dobije unapređenje (npr. prešao je sa pozicije 1 na poziciju 2) koliko minimalno i maksimalno povećanje plate očekuje?
Obrazložiti da li su validne nađene vrednosti. 

Objašnjenje:
Prema trenutnom regresionom modelu, parametar uz promenljivu zvanje je približno ~ 24480 (vidi se iz poziva metode `summary()`). To znači da se `plata` u proseku poveća za 24480$ kada se `zvanje` poveća za 1 (zaposleni je dobio unpaređenje).
Zadatak pita za minimalno i maksimalno povećanje plate, kada se `zvanje` poveća za 1. Zapravo nas pita kolika je minimalna i maksimalna vrednost parametra koji stoji uz `zvanje`. Te dve vrednosti su **interval poverenja** za promenljivu `zvanje`. Treba da nađemo interval poverenja za tu promenljivu. Da nađemo minimlano i maksimalno povećanje plate pozivamo: `get_conf_interval(model, 'zvanje', alpha=0.05)`.
Nakon poziva metode vidimo da je minimalno povećanje plate ~ 14000$, a maksimalno ~ 24000$. 

Ipak, ove vrednosti nisu validne (nisu pouzdane). Teorija kaže da su intervali poverenja validni ako je zadovoljena pretpostavka o nezavisnosti grešaka. U našem primeru pretpostavka nije zadovoljena pa ni nađene vrednosti nisu validne.

In [3]:
# tražimo min i maks vrednosti
min_expected_raise, max_expected_raise = get_conf_interval(model, 'zvanje', alpha=0.05)
print(f'{min_expected_raise:.2f}')
print(f'{max_expected_raise:.2f}')

# provera da li su min i maks vrednosti validne
autocorrelation, _ = independence_of_errors_assumption(model, sm.add_constant(x), y, plot=False)
if autocorrelation is None:
    print('vrednosti su validne, jer je zadovoljena pretpostavka o nezavisnosti gresaka')
else:
    print('vrednosti nisu validne')

13999.72
24289.69
vrednosti nisu validne


**Zadatak 3.** <br>
Unaprediti model tako da su sve pretpostavke zadovoljene i ostvaruje RMSE < `11 170`. Koristiti *train/val* podelu u odnosu: *80/20*. Ispisati meru koju model ostvaruje.

*Napomena: koristiti `random state = 42` (kako bi se rezultati mogli reprodukovati).*

Objašnjenje: 
1. Treba da se setimo naprednijih metoda za nedostajuće vrednosti umesto običnog brisanja redova (koje smo iskoristili na početku). Tako smo ovde koristili interpolaciju jer daje bolje rezultate (uvek treba isprobati različite metode).

2. Treba porazmisliti da li nam je neka kolona suvišna (koristimo vrednosti t-testa). Gledajući vrednosti možemo zaključiti da kolone za pol ne utiču na platu, pa te kolone izbacujemo => i dobijamo bolju RMSE meru.
Takođe, bitno je bar jednu kolonu za pol izbaciti kako ne bismo imali savršenu kolinearnost (kako bi zadovoljili sve pretpostavke, pa tako i pretpostavku o savršenoj kolinearnosti).

3. Podatke iz *train.csv* fajla delimo na trening i validacioni skup pozivom metode `train_test_split`. Bitno je da fitujemo model na trening skupu. Da nađemo najbolji model, gledamo meru nad validacionim skupom gde je cilj da dobijemo što manju RMSE meru.

4. Obavezno prijavljujemo meru na test skupu! To je rezultat našeg modela. Kada smo našli dobre parametre modela, ova mera će biti ispod 11 700 (konkretna vrednost je ~ 11 200).

5. Svaki put kada promenimo parametre modela moramo proveriti da li su sve pretpostavke zadovoljene (tako traži zadatak). U ovom primeru su zadovoljene sve pretpostavke osim pretpostavke o normalnosti grešaka. Ovu pretpostavku možemo zanemariti kada imamo puno podataka (što je ovde i slučaj jer imamo > 150 podataka).

In [4]:
# učitamo podatke
df = pd.read_csv('data/train.csv', sep=',')
# interpolacija umesto brisanja vrednosti zadovoljava sve pretpostavke
df['zvanje'] = df['zvanje'].interpolate(method='spline', order=3, limit_direction='both')
df['godina_doktor'] = df['godina_doktor'].interpolate(method='linear', limit_direction='both')

# brišemo kolonu pol Muski (ili pol Zenski) kako ne bi imali savršenu kolinearnost
df = df.drop(columns=['pol Muski', 'pol Zenski'])

# delimo podatke u odnosu 80-20
x = df.drop(columns=['plata'])
y = df['plata']
x_train, x_val, y_train, y_val = train_test_split(x, y, train_size=0.8, shuffle=True, random_state=42)

# fitujemo model
model = get_fitted_model(x_train,y_train)

# proveravamo da li su zadovoljenepretpostavke (pogledaj objašnjenje)
print(are_assumptions_satisfied(model, x_train, y_train))

# gledamo meru na validacionom skupu kako bi našli najbolji model
val_rmse = get_rmse(model, x_val, y_val)
print(f'validation rmse: {val_rmse:.2f}')

# prijavljujemo meru na test skupu
df_test = pd.read_csv('data/test.csv', sep=',')
x_test = df_test.drop(columns=['plata', 'pol Muski', 'pol Zenski'])
y_test = df_test['plata']
test_rmse = get_rmse(model, x_test, y_test)
print(f'test rmse: {test_rmse:.2f}')

False
validation rmse: 21029.75
test rmse: 11165.79


**Zadatak 4.** <br>
Da li je bar jedna nezavisna promenljiva u linearnoj vezi sa platom? Obrazloži odgovor.


In [5]:
# pozivom model.summary() vidimo da je p-vrednost F-testa 2.44e-30 što je manje od 5%.
# Zato smo 95% sigurni da bar jedna nezavisna promenljiva ima linearnu vezu sa platom.
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  plata   R-squared:                       0.463
Model:                            OLS   Adj. R-squared:                  0.454
Method:                 Least Squares   F-statistic:                     50.04
Date:                Tue, 28 Nov 2023   Prob (F-statistic):           2.52e-30
Time:                        18:16:43   Log-Likelihood:                -2688.8
No. Observations:                 237   AIC:                             5388.
Df Residuals:                     232   BIC:                             5405.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            4.407e+04   5502.727     

**Zadatak 5.** <br>
Objasni meru prilagodjeni $r^2$.

In [6]:
# Ovo pitanje je čisto teorijsko pitanje. Odgovor na pitanje se može pronaći 
# u materijalima za predavanja i vezbe kada se obradjivala visestruka regresija.